<a href="https://colab.research.google.com/github/codebjjung/ML-DL/blob/main/CNN_seq_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# libraries

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import os

In [5]:
# set hyper parameters

learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [6]:
# make a data pipelining

mnist = keras.datasets.mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.astype(np.float32) / 255.
test_images = test_images.astype(np.float32) / 255.
train_images = np.expand_dims(train_images, axis = -1)
test_images = np.expand_dims(test_images, axis = -1)

train_labels = to_categorical(train_labels, 10)
test_labels = to_categorical(test_labels, 10)

train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).shuffle(buffer_size=100000).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).shuffle(buffer_size=100000).batch(batch_size)

11490434/11490434 [==============================] - 0s 0us/step


In [9]:
def create_model():
  model = keras.Sequential()
  # Conv
  model.add(keras.layers.Conv2D(filters=32, kernel_size=3, activation=tf.nn.relu, padding='SAME', input_shape=(28,28,1)))

  # Max pooling
  model.add(keras.layers.MaxPool2D(padding='SAME'))

  # Conv
  model.add(keras.layers.Conv2D(filters=64, kernel_size=3, activation=tf.nn.relu, padding='SAME'))

  # Max pooling
  model.add(keras.layers.MaxPool2D(padding='SAME'))

  # Conv
  model.add(keras.layers.Conv2D(filters=128, kernel_size=3, activation=tf.nn.relu, padding='SAME'))

  # Max pooling
  model.add(keras.layers.MaxPool2D(padding='SAME'))

  # Fully connected layer
  model.add(keras.layers.Flatten())
  model.add(keras.layers.Dense(256,activation=tf.nn.relu))
  model.add(keras.layers.Dropout(0.4))
  model.add(keras.layers.Dense(10))
  return model

In [10]:
model = create_model()
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 28, 28, 32)        320       
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 14, 14, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_4 (Conv2D)           (None, 14, 14, 64)        18496     
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 7, 7, 64)          0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 7, 7, 128)         73856     
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 4, 4, 128)        